# Feature Selection - correlation-based  

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
import time
from sklearn.feature_selection import RFECV
from scipy.stats.contingency import association
import os
import random

Import of 'ff_train_agg' dataset from check point 2:

In [2]:
ff_train_agg = pd.read_csv("C:/Users/Avishai/Documents/General/Personal/DS Course/Google drive content/Projects/Final project-ASHRAE-Great Energy Predictor III/Data/CheckPoint2/ff_train_agg.csv")
ff_train_agg

,site_id,building_id,date,time_range_x_mode,time_range_y_mode,meter_reading_sum,precip_depth_sum,precip_depth_min,precip_depth_max,precip_depth_sd,...,dew_temperature_sd_sqrt,sea_level_pressure_sd_sqrt,rel_humid_sd_sqrt,is_missing_year_built,is_missing_sea_level_pressure_mean,is_missing_sea_level_pressure_min,is_missing_sea_level_pressure_max,is_missing_dew_temperature_sd_sqrt,is_missing_sea_level_pressure_sd_sqrt,is_missing_rel_humid_sd_sqrt
0,0,103,2016-02-29,-0.5,-0.866,3672.32,0,0,0,0,...,3,4,4,0,0,0,0,0,0,0
1,0,103,2016-02-29,1.0,0.000,1350.74,0,0,0,0,...,4,3,4,0,0,0,0,0,0,0
2,0,103,2016-03-01,-0.5,0.866,1266.30,0,0,0,0,...,3,2,2,0,0,0,0,0,0,0
3,0,103,2016-03-01,1.0,0.000,1604.00,0,0,0,0,...,4,2,4,0,0,0,0,0,0,0
4,0,103,2016-03-02,-0.5,0.866,1604.00,0,0,0,0,...,2,2,2,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418846,15,1413,2016-12-29,-0.5,-0.866,30.26,1,0,2,3,...,3,4,2,0,0,0,0,0,0,0
418847,15,1413,2016-12-29,1.0,0.000,26.20,0,0,0,0,...,3,2,3,0,0,0,0,0,0,0
418848,15,1413,2016-12-30,-0.5,0.866,28.67,0,0,0,0,...,2,2,2,0,0,0,0,0,0,0
418849,15,1413,2016-12-31,-0.5,0.866,27.89,0,0,0,0,...,2,1,1,0,0,0,0,0,0,0


The nominal variables will be factorized later on - right before execution of Cramer's v calculation. I don't factorize them now, as all the nominal binary variables can be considered both nominal and ordinal (with a single distance between values). As ordinal, they can also be used in 'spearmanr' function, which doesn't accept categorical variables as input. Since that, I am avoiding factorizing them at this point and wait for 'spearmanr' to be permormed first.  

## Univariable Analysis

I start with filter method, to check correlations within X variables, and between X variables to the outcome variable.
One variable in each pair of X variables that exhibits an absolute correlation of 0.7 or higher will be eliminated. The variable to be dropped in each pair is the one with a weaker correlation with the outcome variable.

'Table1' from 'pyMechkar' package is good for a group check, therefore it can be used for categorical outcome only. 
For contiuous outcome, I run correlations, using 'for' loop. The same as I did in the EDA.

In [3]:
# A check for missing values in the dataset, prior to correlation calculation
has_missing_values = ff_train_agg.isnull().any().any()

if has_missing_values:
    print("The dataset contains missing values.")
else:
    print("The dataset does not contain any missing values.")

The dataset does not contain any missing values.


I will define 2 groups: 

A - continuous and ordinal (including binary) variables - will be executed by the 'spearmanr' function.

B - nominal (including binary) variables - will be executed by "Cramer's V" method.

### A. Correlations - continuous and ordinal (including binary) variables:

#### Correlations between variables  of group A and themselves:

In [4]:
# Identifying string variables to be excluded from the correlation calculation
string_vars = ff_train_agg.select_dtypes(include = ['object']).columns.tolist()

print(len(string_vars))
string_vars

1


['date']

To be on the safe side, I use the assumptions of 'Spearman' correlation analysis, for a case which the data is not normally distributed.

#### 'spearmanr' function:

In [5]:
''' Exclude columns that are not in group A, which are the nominal without the binary i.e.: 'site_id', 'building_id' and all columns that 
    were factorized earlier in R, following the 2 categorizations have carried out in the 'data cleansing' stage. '''
exclude = ["site_id", "building_id", "date", "precip_depth_sum", "precip_depth_min", "precip_depth_max", "precip_depth_sd", "rel_humid_max", "elevation_mode", "year_built"]     

# Variables for correlation test by 'spearmanr' function
vars_for_spearmanr = [var for var in ff_train_agg.columns if var not in exclude]

# A subset of the relevant columns for the correlation test
selected_cols = ff_train_agg.loc[:, vars_for_spearmanr]

# Calculate Spearman correlation matrix and p-values
corr_matrix, p_values = stats.spearmanr(selected_cols, nan_policy = 'omit')

# It's better to create an empty list than an empty dataframe because of the 'append' used later
correlations_df_A = []

# Iterate over the correlation matrix and p-values
# 'enumerate' retrieves both, an index and its corresponding element out of a sequence
for i, n1 in enumerate(vars_for_spearmanr):
    for j, n2 in enumerate(vars_for_spearmanr):
        # Skips cases to avoid duplicate pairs and a correlation between a variable to itself
        # Actually it's implemented on the cases where j > i
        if j <= i:
            continue
        correlation = corr_matrix[i, j]
        p_value = p_values[i, j]
        correlations_df_A.append([n1, n2, correlation, p_value])

# The results dataframe
correlations_df_A = pd.DataFrame(correlations_df_A, columns = ["Variable1", "Variable2", "Correlation", "P_value"])
correlations_df_A

,Variable1,Variable2,Correlation,P_value
0,time_range_x_mode,time_range_y_mode,-0.015446,1.572204e-23
1,time_range_x_mode,meter_reading_sum,0.050030,2.866644e-230
2,time_range_x_mode,air_temperature_mean,0.138115,0.000000e+00
3,time_range_x_mode,air_temperature_min,0.132163,0.000000e+00
4,time_range_x_mode,air_temperature_max,0.147599,0.000000e+00
...,...,...,...,...
2273,is_missing_sea_level_pressure_max,is_missing_sea_level_pressure_sd_sqrt,0.059919,0.000000e+00
2274,is_missing_sea_level_pressure_max,is_missing_rel_humid_sd_sqrt,0.003669,1.758583e-02
2275,is_missing_dew_temperature_sd_sqrt,is_missing_sea_level_pressure_sd_sqrt,0.096037,0.000000e+00
2276,is_missing_dew_temperature_sd_sqrt,is_missing_rel_humid_sd_sqrt,0.235335,0.000000e+00


#### Pairs of group A variables that have an absolute correlation of 0.7 or higher and statistically significant: 

In [6]:
# Prevent the outcome variable to be presented in this table, as here I want to see the undesired high correlations between X variables
high_correlations_df_A = correlations_df_A[(abs(correlations_df_A['Correlation']) >= 0.7) & (correlations_df_A['P_value'] <= 0.05) & 
                                                            (~correlations_df_A['Variable1'].str.contains('meter_reading_sum')) &
                                                            (~correlations_df_A['Variable2'].str.contains('meter_reading_sum'))]

print(len(high_correlations_df_A))
high_correlations_df_A

29


,Variable1,Variable2,Correlation,P_value
102,time_range_y_mode,is_working_hours,-0.863028,0.0
198,air_temperature_mean,air_temperature_min,0.970347,0.0
199,air_temperature_mean,air_temperature_max,0.986995,0.0
262,air_temperature_min,air_temperature_max,0.927746,0.0
448,dew_temperature_mean,dew_temperature_min,0.971144,0.0
449,dew_temperature_mean,dew_temperature_max,0.967547,0.0
508,dew_temperature_min,dew_temperature_max,0.941857,0.0
625,sea_level_pressure_mean,sea_level_pressure_min,0.944530,0.0
626,sea_level_pressure_mean,sea_level_pressure_max,0.963113,0.0
682,sea_level_pressure_min,sea_level_pressure_max,0.916193,0.0


Filtering 'correlations_df_A' to exhibit only rows contain the outcome variable. I will be assisted this table to drop variables:

In [7]:
outcome_correlations_df_A = correlations_df_A[(correlations_df_A['Variable1'] == "meter_reading_sum") | (correlations_df_A['Variable2'] == "meter_reading_sum")]

print(len(outcome_correlations_df_A))
outcome_correlations_df_A

67


,Variable1,Variable2,Correlation,P_value
1,time_range_x_mode,meter_reading_sum,0.050030,2.866644e-230
67,time_range_y_mode,meter_reading_sum,-0.096596,0.000000e+00
133,meter_reading_sum,air_temperature_mean,0.396205,0.000000e+00
134,meter_reading_sum,air_temperature_min,0.399137,0.000000e+00
135,meter_reading_sum,air_temperature_max,0.382949,0.000000e+00
...,...,...,...,...
193,meter_reading_sum,is_missing_sea_level_pressure_min,-0.050228,4.418281e-232
194,meter_reading_sum,is_missing_sea_level_pressure_max,-0.048618,1.455379e-217
195,meter_reading_sum,is_missing_dew_temperature_sd_sqrt,0.001595,3.018722e-01
196,meter_reading_sum,is_missing_sea_level_pressure_sd_sqrt,-0.055916,3.352597e-287


Here it will be determined among all group A variables, which variable in each highly correlated pair, will be removed from the dataset.
Either Variable1 or Variable2, that has the lower correlation with the outcome variable will be dropped from the dataset.

In [8]:
# Add "Drop" column to 'high_correlations_df_A1'
high_correlations_df_A['Drop'] = ''

# Iterate over each row in 'high_correlations_df_A'
# 'index' - the index value of the current row being iterated
# 'row' - the content of the row being iterated
for index, row in high_correlations_df_A.iterrows():
    var1 = row['Variable1']
    var2 = row['Variable2']
    
    # Extract the correlations of both Variable1 and Variable2 with the outcome varialbe out of 'outcome_correlations_df_A'
    corr_var1_outcome = outcome_correlations_df_A.loc[(outcome_correlations_df_A['Variable1'] == var1) | (outcome_correlations_df_A['Variable2'] == var1), 'Correlation'].values[0]
    corr_var2_outcome = outcome_correlations_df_A.loc[(outcome_correlations_df_A['Variable1'] == var2) | (outcome_correlations_df_A['Variable2'] == var2), 'Correlation'].values[0]
    
    p_value_var1_outcome = outcome_correlations_df_A.loc[(outcome_correlations_df_A['Variable1'] == var1) | (outcome_correlations_df_A['Variable2'] == var1), 'P_value'].values[0]
    p_value_var2_outcome = outcome_correlations_df_A.loc[(outcome_correlations_df_A['Variable1'] == var2) | (outcome_correlations_df_A['Variable2'] == var2), 'P_value'].values[0]
    
    print(var1, ' ', var2)
    print(corr_var1_outcome, ' ', corr_var2_outcome)

    # Discover which variable has the lower correlation with the outcome variable, and assign it to "Drop" column
    #  Use results of 'outcome_correlations_df_A' based on sifnificant p values only
    if (p_value_var1_outcome <= 0.05) and (p_value_var2_outcome <= 0.05):
        if (abs(corr_var1_outcome) < abs(corr_var2_outcome)):
            high_correlations_df_A.at[index, 'Drop'] = var1
        else:
            high_correlations_df_A.at[index, 'Drop'] = var2
    else:
        print(var1, ' ', var2)
        print('non significant p value in correlation with the outcome variable')

print('\n', len(high_correlations_df_A))
high_correlations_df_A

time_range_y_mode   is_working_hours
-0.09659567043528187   0.05884277518443084
air_temperature_mean   air_temperature_min
0.39620474770504505   0.39913723366958426
air_temperature_mean   air_temperature_max
0.39620474770504505   0.38294887845973863
air_temperature_min   air_temperature_max
0.39913723366958426   0.38294887845973863
dew_temperature_mean   dew_temperature_min
0.3762308646143151   0.36643434848549505
dew_temperature_mean   dew_temperature_max
0.3762308646143151   0.38775330835011623
dew_temperature_min   dew_temperature_max
0.36643434848549505   0.38775330835011623
sea_level_pressure_mean   sea_level_pressure_min
-0.06979175996709094   -0.06690892377063862
sea_level_pressure_mean   sea_level_pressure_max
-0.06979175996709094   -0.07831308889485429
sea_level_pressure_min   sea_level_pressure_max
-0.06690892377063862   -0.07831308889485429
wind_speed_mean   wind_speed_min
-0.0239583811968351   -0.01955023169178979
wind_speed_mean   wind_speed_max
-0.0239583811968351   -0.02

C:\Users\Avishai\AppData\Local\Temp\ipykernel_6312\1504905090.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_correlations_df_A['Drop'] = ''


,Variable1,Variable2,Correlation,P_value,Drop
102,time_range_y_mode,is_working_hours,-0.863028,0.0,is_working_hours
198,air_temperature_mean,air_temperature_min,0.970347,0.0,air_temperature_mean
199,air_temperature_mean,air_temperature_max,0.986995,0.0,air_temperature_max
262,air_temperature_min,air_temperature_max,0.927746,0.0,air_temperature_max
448,dew_temperature_mean,dew_temperature_min,0.971144,0.0,dew_temperature_min
449,dew_temperature_mean,dew_temperature_max,0.967547,0.0,dew_temperature_mean
508,dew_temperature_min,dew_temperature_max,0.941857,0.0,dew_temperature_min
625,sea_level_pressure_mean,sea_level_pressure_min,0.944530,0.0,sea_level_pressure_min
626,sea_level_pressure_mean,sea_level_pressure_max,0.963113,0.0,sea_level_pressure_mean
682,sea_level_pressure_min,sea_level_pressure_max,0.916193,0.0,sea_level_pressure_min


In [9]:
# Chek rows where 'Drop' is empty. It might happen when p value of one or both of the variables is not significant 
high_correlations_df_A[(high_correlations_df_A['Drop'] == '')]

,Variable1,Variable2,Correlation,P_value,Drop
1003,rel_humid_mean,rel_humid_min,0.930789,0.0,


In 'high_correlations_df_A', there is 1 row where the 'Drop' column contains an empty string (''). This occurs in pair:'rel_humid_mean', 'rel_humid_min'. The reason is that one of the variables, or both, had a non-significant 
p value in its correlation with the outcome variable, resulted in 'Drop' column not being populated. 

A decision needs to be made about which variable will be dropped from the dataset. Lets see what are the correlations and the 
p values of each of those variables with the outcome variable.

In [10]:
outcome_correlations_df_A[(outcome_correlations_df_A['Variable2'] == 'rel_humid_mean') | (outcome_correlations_df_A['Variable2'] == 'rel_humid_min')]

,Variable1,Variable2,Correlation,P_value
147,meter_reading_sum,rel_humid_mean,0.002170,1.601662e-01
148,meter_reading_sum,rel_humid_min,-0.014312,1.985509e-20


The absolute correlation value between "rel_humid_min" and the outcome variable is significantly higher (in one magnitude) compared to that of "rel_humid_mean". Although the p-value of "rel_humid_mean" is not statistically significant, the substantial difference in the absolute correlation values in favor of "rel_humid_min" leads me to the decision to drop "rel_humid_mean".

In [11]:
# Variables to be dropped based on high correlations between X variables
drop_variables_A = high_correlations_df_A['Drop'].unique().tolist()
# Append 'rel_humid_mean' and 'year_built' to the list
drop_variables_A.append('rel_humid_mean')

# Remove any empty strings ('') from the list, if they exist
if '' in drop_variables_A:
    drop_variables_A.remove('')
    
print(len(drop_variables_A))
drop_variables_A

21


['is_working_hours',
 'air_temperature_mean',
 'air_temperature_max',
 'dew_temperature_min',
 'dew_temperature_mean',
 'sea_level_pressure_min',
 'sea_level_pressure_mean',
 'wind_speed_min',
 'wind_speed_max',
 'cloud_coverage_max',
 'day_of_week_x',
 'month_x',
 'day_of_year_y',
 'is_upward_date',
 'season_x',
 'is_zero_precip_depth_sum',
 'is_zero_precip_depth_max',
 'is_zero_precip_depth_sd',
 'is_missing_sea_level_pressure_mean',
 'is_missing_sea_level_pressure_max',
 'rel_humid_mean']

### B. Correlations - nominal (including binary) variables:

#### Correlations between variables of group B and themselves:

In [12]:
# Nominal variables (not including "site_id" and 'building_id' that are not relevant for the correlation test)
""" "precip_depth_sum", "precip_depth_min", "precip_depth_max", "precip_depth_sd", "rel_humid_max", "elevation_mode" and "year_built" are columns 
     that were factorized earlier in 'Data Cleansing' notebook, following categorization has carried out as part of the outlier and missingness handling. """ 
nominal = ["precip_depth_sum", "precip_depth_min", "precip_depth_max", "precip_depth_sd", "rel_humid_max", "elevation_mode", "year_built"] + [var for var in ff_train_agg.columns if var.startswith(("sw", "is"))]

print(len(nominal))
nominal

39


['precip_depth_sum',
 'precip_depth_min',
 'precip_depth_max',
 'precip_depth_sd',
 'rel_humid_max',
 'elevation_mode',
 'year_built',
 'sw_primary_use_Education',
 'sw_primary_use_Entertainment_publicAssembly',
 'sw_primary_use_FoodSalesAndService',
 'sw_primary_use_Healthcare',
 'sw_primary_use_Lodging_residential',
 'sw_primary_use_Manufacturing_industrial',
 'sw_primary_use_Office',
 'sw_primary_use_Other',
 'sw_primary_use_Parking',
 'sw_primary_use_PublicServices',
 'sw_primary_use_ReligiousWorship',
 'sw_primary_use_Retail',
 'sw_primary_use_TechnologyScience',
 'sw_primary_use_Utility',
 'is_working_hours',
 'is_weekend',
 'is_holiday',
 'is_upward_date',
 'is_precip_dew_temperature_mean',
 'is_precip_air_temperature_mean',
 'is_zero_precip_depth_sum',
 'is_zero_precip_depth_min',
 'is_zero_precip_depth_max',
 'is_zero_precip_depth_sd',
 'is_even_cloud_coverage',
 'is_missing_year_built',
 'is_missing_sea_level_pressure_mean',
 'is_missing_sea_level_pressure_min',
 'is_missing_

In [13]:
ff_train_agg.dtypes

site_id                                    int64
building_id                                int64
date                                      object
time_range_x_mode                        float64
time_range_y_mode                        float64
                                          ...   
is_missing_sea_level_pressure_min          int64
is_missing_sea_level_pressure_max          int64
is_missing_dew_temperature_sd_sqrt         int64
is_missing_sea_level_pressure_sd_sqrt      int64
is_missing_rel_humid_sd_sqrt               int64
Length: 78, dtype: object

In [14]:
# Before applying 'Cramer's V,' transform nominal columns into the 'categorical' format
ff_train_agg[nominal] = ff_train_agg[nominal].astype("category")

In [15]:
ff_train_agg.dtypes

site_id                                     int64
building_id                                 int64
date                                       object
time_range_x_mode                         float64
time_range_y_mode                         float64
                                           ...   
is_missing_sea_level_pressure_min        category
is_missing_sea_level_pressure_max        category
is_missing_dew_temperature_sd_sqrt       category
is_missing_sea_level_pressure_sd_sqrt    category
is_missing_rel_humid_sd_sqrt             category
Length: 78, dtype: object

In [16]:
vars_for_cramers_v = nominal

# Check if any variable in 'vars_for_cramers_v' is not categorical
any_not_categorical = any(not pd.api.types.is_categorical_dtype(ff_train_agg[var]) for var in vars_for_cramers_v)

if any_not_categorical:
    print("There is at least one variable in 'vars_for_cramers_v' that is not categorical.")
else:
    print("All variables in 'vars_for_cramers_v' are categorical.")

All variables in 'vars_for_cramers_v' are categorical.


##### "Cramer's v":

In [17]:
# Initialize an empty list to store results
correlations_df_B = []

# Nested loop to calculate Cramer's V for all pairs of columns
# 'enumerate' retrieves index and content of a sequence
for i, var1 in enumerate(vars_for_cramers_v):
    for var2 in vars_for_cramers_v[i+1:]:
        # Create a contingency table for the two categorical variables
        contingency_table = pd.crosstab(ff_train_agg[var1], ff_train_agg[var2])
        
        # Calculate the chi-squared statistic, p-value, degrees of freedom, and expected frequencies
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        
        # Calculate Cramer's V
        n = contingency_table.sum().sum()
        cramers_v = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))
        
        # Append results to the correlations_df_A2 list
        correlations_df_B.append({'Variable1': var1, 'Variable2': var2, "Cramer's V": cramers_v, 'P_value': p})

# Convert the list of dictionaries to a DataFrame
correlations_df_B = pd.DataFrame(correlations_df_B)
correlations_df_B

,Variable1,Variable2,Cramer's V,P_value
0,precip_depth_sum,precip_depth_min,0.623576,0.000000
1,precip_depth_sum,precip_depth_max,0.671847,0.000000
2,precip_depth_sum,precip_depth_sd,0.500488,0.000000
3,precip_depth_sum,rel_humid_max,0.251675,0.000000
4,precip_depth_sum,elevation_mode,0.281726,0.000000
...,...,...,...,...
736,is_missing_sea_level_pressure_max,is_missing_sea_level_pressure_sd_sqrt,0.059853,0.000000
737,is_missing_sea_level_pressure_max,is_missing_rel_humid_sd_sqrt,0.003557,0.021315
738,is_missing_dew_temperature_sd_sqrt,is_missing_sea_level_pressure_sd_sqrt,0.096002,0.000000
739,is_missing_dew_temperature_sd_sqrt,is_missing_rel_humid_sd_sqrt,0.235277,0.000000


#### Pairs of nominal variables that have an absolute measure of association of 0.7 or higher and statistically significant:

In [18]:
high_correlations_df_B = correlations_df_B[(abs(correlations_df_B["Cramer's V"]) >= 0.7) & (correlations_df_B["P_value"] <= 0.05)]

print(len(high_correlations_df_B))
high_correlations_df_B

23


,Variable1,Variable2,Cramer's V,P_value
26,precip_depth_sum,is_zero_precip_depth_sum,1.000000,0.0
27,precip_depth_sum,is_zero_precip_depth_min,0.808466,0.0
28,precip_depth_sum,is_zero_precip_depth_max,1.000000,0.0
29,precip_depth_sum,is_zero_precip_depth_sd,0.855237,0.0
63,precip_depth_min,is_zero_precip_depth_sum,0.732432,0.0
64,precip_depth_min,is_zero_precip_depth_min,1.000000,0.0
65,precip_depth_min,is_zero_precip_depth_max,0.732432,0.0
99,precip_depth_max,is_zero_precip_depth_sum,1.000000,0.0
100,precip_depth_max,is_zero_precip_depth_min,0.738423,0.0
101,precip_depth_max,is_zero_precip_depth_max,1.000000,0.0


'Cramer's v' retrieves 23 pairs of nominal variables that have a statistically significant absolute measure of association of 0.7 or higher.

Correlations between group B (nominal) variables to the outcome variable can't be calculated (in order to decide which of the variables in a pair will be dropped) since the outcome variable is continuous, thus cannot be included in the calculation using "Cramer's V" that receives categorical variables only. Therefore decision about the variable to be drpped will be random.

In [19]:
# Initialize an empty set to store variables to be dropped
# set ensure that duplicated variables will not be included
drop_variables_B = set()

# Iterate through rows in 'high_correlations_df_B'
for index, row in high_correlations_df_B.iterrows():
    variable1 = row['Variable1']
    variable2 = row['Variable2']
    
    # Check if either variable is in 'drop_variables_A'
    if variable1 in drop_variables_A or variable2 in drop_variables_A:
        # If either is in 'drop_variables_A', no action needed for this pair
        continue
    
    # Randomly select one of the variables in the pair
    selected_variable = random.choice([variable1, variable2])
    
    # Add the selected variable to 'drop_variables_B' (a set)
    drop_variables_B.add(selected_variable)
    
# Convert 'drop_variables_B' back to a list if needed
drop_variables_B = list(drop_variables_B)

print(len(drop_variables_B))
print(drop_variables_B)

3
['year_built', 'precip_depth_sum', 'is_zero_precip_depth_min']


In [20]:
drop_variables_correlations = drop_variables_A + drop_variables_B

print(len(drop_variables_correlations))
drop_variables_correlations

24


['is_working_hours',
 'air_temperature_mean',
 'air_temperature_max',
 'dew_temperature_min',
 'dew_temperature_mean',
 'sea_level_pressure_min',
 'sea_level_pressure_mean',
 'wind_speed_min',
 'wind_speed_max',
 'cloud_coverage_max',
 'day_of_week_x',
 'month_x',
 'day_of_year_y',
 'is_upward_date',
 'season_x',
 'is_zero_precip_depth_sum',
 'is_zero_precip_depth_max',
 'is_zero_precip_depth_sd',
 'is_missing_sea_level_pressure_mean',
 'is_missing_sea_level_pressure_max',
 'rel_humid_mean',
 'year_built',
 'precip_depth_sum',
 'is_zero_precip_depth_min']

Dropping column from the dataset as a result of high correlation between 2 variables

In [21]:
ff_train_agg.drop(columns = drop_variables_correlations, inplace = True)
ff_train_agg

,site_id,building_id,date,time_range_x_mode,time_range_y_mode,meter_reading_sum,precip_depth_min,precip_depth_max,precip_depth_sd,air_temperature_min,...,is_precip_air_temperature_mean,is_even_cloud_coverage,dew_temperature_sd_sqrt,sea_level_pressure_sd_sqrt,rel_humid_sd_sqrt,is_missing_year_built,is_missing_sea_level_pressure_min,is_missing_dew_temperature_sd_sqrt,is_missing_sea_level_pressure_sd_sqrt,is_missing_rel_humid_sd_sqrt
0,0,103,2016-02-29,-0.5,-0.866,3672.32,0,0,0,13.3,...,1,1,3,4,4,0,0,0,0,0
1,0,103,2016-02-29,1.0,0.000,1350.74,0,0,0,16.7,...,1,1,4,3,4,0,0,0,0,0
2,0,103,2016-03-01,-0.5,0.866,1266.30,0,0,0,13.9,...,1,1,3,2,2,0,0,0,0,0
3,0,103,2016-03-01,1.0,0.000,1604.00,0,0,0,17.8,...,1,1,4,2,4,0,0,0,0,0
4,0,103,2016-03-02,-0.5,0.866,1604.00,0,0,0,14.4,...,1,1,2,2,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418846,15,1413,2016-12-29,-0.5,-0.866,30.26,0,2,3,-1.7,...,1,1,3,4,2,0,0,0,0,0
418847,15,1413,2016-12-29,1.0,0.000,26.20,0,0,0,-1.7,...,1,1,3,2,3,0,0,0,0,0
418848,15,1413,2016-12-30,-0.5,0.866,28.67,0,0,0,-1.7,...,1,1,2,2,2,0,0,0,0,0
418849,15,1413,2016-12-31,-0.5,0.866,27.89,0,0,0,-4.4,...,1,0,2,1,1,0,0,0,0,0


Export 'ff_train_agg' dataset to Check point 3:

In [22]:
file_path = "C:/Users/Avishai/Documents/General/Personal/DS Course/Google drive content/Projects/Final project-ASHRAE-Great Energy Predictor III/Data/CheckPoint3/ff_train_agg.csv"
ff_train_agg.to_csv(file_path, index = False)